# Conenction Interface
DocTable2 provides several options for managing database connections. You can either (1) create a persistent database connection upon instantiation using `persisten_conn=True` (default), (2) create a new database connection each time a query is executed (`persistent_conn=False`), or (3) provide an external connection for each individual query by using the keyword argument `conn=<sqlalchemy connection object>` to any DocTable2 that performs a query (most commonly `.select()`, `.update()`, and `.delete()`).

In [26]:
import time
from multiprocessing import Process
import os
import sys
sys.path.append('..')
import doctable as dt

## Persistent Connection
This mode is most useful when you would like to lock a database from being written to at the same time by multiple threads. A write operation from one thread will lock the database for a period of time such that **other connections will throw exceptions if they try to read or write in that time period.** This is the default behavior because there is some time cost to opening a database connection on every query, so this mode allows you to avoid this overhead.

See [this stackoverflow questions about concurrent sqlite connections](https://stackoverflow.com/questions/10325683/can-i-read-and-write-to-a-sqlite-database-concurrently-from-multiple-connections) for more detail.

This case is difficult to write an example case for, so for now I'll just leave it with this note.

In [ ]:
def thread_writer(schema,fname):
    db = DocTable2(schema, persistent_conn=True, fname=fname)
    for i in range(100000):
        db.insert({'id':i,'value':i/2})
        #time.sleep(0.001) # in seconds

def thread_writer2(schema,fname):
    db = dt.DocTable2(schema, persistent_conn=True, fname=fname)
    for i in range(1000000):
        db.insert({'id':i,'value':i/2})
        #time.sleep(0.001) # in seconds

fname = 'tmp_connections.db'
schema = (('id', 'integer'), ('value','float'))
p_writer = Process(target=thread_writer, args=(schema,fname))
p_writer2 = Process(target=thread_writer2, args=(schema,fname))

p_writer.start()
p_writer2.start()

p_writer.join()
p_writer2.join()
db = DocTable2(schema, fname)
print(db)

In [ ]:
## Non-Persistent Connection
This mode is most useful when attempting to insert items from the same database 

In [24]:


[db.insert({'id':i,'value':i/2}) for i in range(5)]
db.select()

[(0, 0.0),
 (1, 0.5),
 (2, 1.0),
 (3, 1.5),
 (4, 2.0),
 (0, 0.0),
 (1, 0.5),
 (2, 1.0),
 (3, 1.5),
 (4, 2.0),
 (0, 0.0),
 (1, 0.5),
 (2, 1.0),
 (3, 1.5),
 (4, 2.0)]

In [22]:
db2 = dt.DocTable2(fname=fname)

In [23]:
db2

In [16]:
db.select()

[(0, 0.0), (1, 1.5), (2, 3.0), (3, 4.5), (4, 6.0)]

In [ ]:
# custom class that will delete it's database file upon deletion
class TempDocTable2(dt.DocTable2):
    def __delete__():
        super().__delete__()
        if os.path.exists(self._tabname):
            os.remove(self._tabname)